In [1]:
import os
os.chdir('../')

import pandas as pd
import numpy as np

data = pd.read_csv("Datasets/analysis_data.csv")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# ------------------------------
# Load Data
# ------------------------------
df = data.copy()
y = df["monthly_spend"]
X = df.drop(columns=["monthly_spend"])

# ------------------------------
# Train-test split
# ------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ------------------------------
# Identify column types
# ------------------------------
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# ------------------------------
# Preprocessing: impute + encode
# ------------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), cat_cols),
        ("num", SimpleImputer(strategy="median"), num_cols)
    ]
)

X_train_p = preprocessor.fit_transform(X_train)
X_test_p = preprocessor.transform(X_test)

# ----------------------------------------------------------
# 🚀 1. LightGBM (SOTA for tabular data)
# ----------------------------------------------------------
from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(
    n_estimators=1500,
    learning_rate=0.02,
    max_depth=-1,
    num_leaves=40,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1.0,
    reg_lambda=1.0,
    random_state=42
)

lgbm.fit(X_train_p, y_train)
pred_lgbm = lgbm.predict(X_test_p)
rmse_lgbm = mean_squared_error(y_test, pred_lgbm, squared=False)

print('----------------------------------------------------------------')
print('LGBM:',rmse_lgbm)
print('----------------------------------------------------------------')

# ----------------------------------------------------------
# 🚀 2. XGBoost (classic boosted trees)
# ----------------------------------------------------------
from xgboost import XGBRegressor

xgb = XGBRegressor(
    n_estimators=1500,
    learning_rate=0.03,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1.0,
    reg_lambda=1.0,
    objective="reg:squarederror",
    random_state=42
)

xgb.fit(X_train_p, y_train)
pred_xgb = xgb.predict(X_test_p)
rmse_xgb = mean_squared_error(y_test, pred_xgb, squared=False)

print('----------------------------------------------------------------')
print('XGB:',rmse_xgb)
print('----------------------------------------------------------------')

# ----------------------------------------------------------
# FIX categorical columns for CatBoost
# ----------------------------------------------------------
X_train_cat = X_train.copy()
X_test_cat = X_test.copy()

# Convert NaN categorical values to string "Unknown"
X_train_cat[cat_cols] = X_train_cat[cat_cols].astype(str).fillna("Unknown")
X_test_cat[cat_cols] = X_test_cat[cat_cols].astype(str).fillna("Unknown")

cat_idx = [X.columns.get_loc(c) for c in cat_cols]

from catboost import CatBoostRegressor

cat = CatBoostRegressor(
    iterations=1500,
    learning_rate=0.03,
    depth=8,
    l2_leaf_reg=3.0,
    loss_function='RMSE',
    random_seed=42,
    verbose=False
)

cat.fit(X_train_cat, y_train, cat_features=cat_idx)

pred_cat = cat.predict(X_test_cat)
rmse_cat = mean_squared_error(y_test, pred_cat, squared=False)

print(f"⭐ CatBoost RMSE: {rmse_cat:.4f}")


# ----------------------------------------------------------
# FINAL COMPARISON
# ----------------------------------------------------------
print("🔥 STATE-OF-THE-ART MODEL PERFORMANCE (RMSE):")
print(f"⭐ LightGBM: {rmse_lgbm:.4f}")
print(f"⭐ XGBoost:   {rmse_xgb:.4f}")
print(f"⭐ CatBoost:  {rmse_cat:.4f}")

best_model = min(
    [("LightGBM", rmse_lgbm), ("XGBoost", rmse_xgb), ("CatBoost", rmse_cat)],
    key=lambda x: x[1]
)

print("\n🏆 BEST MODEL SO FAR:", best_model[0], "with RMSE =", best_model[1])


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002835 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1791
[LightGBM] [Info] Number of data points in the train set: 32000, number of used features: 22
[LightGBM] [Info] Start training from score 1665.647021


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


----------------------------------------------------------------
LGBM: 256.17664189023856
----------------------------------------------------------------


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


----------------------------------------------------------------
XGB: 259.661225798345
----------------------------------------------------------------


CatBoostError: Invalid type for cat_feature[non-default value idx=63,feature_idx=4]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.

⭐ CatBoost RMSE: 254.4923


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


CATBOOST optimization

In [7]:
# ============================================
# ADVANCED CATBOOST PIPELINE TO BEAT ELASTICNET
# ============================================

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

# You may need to install these if not present:
# !pip install catboost optuna

from catboost import CatBoostRegressor
import optuna

# --------------------------------------------
# 1. Load data and basic setup
# --------------------------------------------
df = data.copy()  # assuming `data` is your cleaned dataset
target_col = "monthly_spend"

y = df[target_col]
X = df.drop(columns=[target_col])

# --------------------------------------------
# 2. Identify column types
# --------------------------------------------
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Categorical columns:", cat_cols)
print("Numeric columns:", num_cols)

# --------------------------------------------
# 3. Light feature engineering (boosting-friendly)
#    - log1p for positive numeric columns (reduces skew)
# --------------------------------------------
X_fe = X.copy()

for col in num_cols:
    # Only log-transform strictly positive columns
    if (X_fe[col] > 0).all():
        X_fe[f"{col}_log1p"] = np.log1p(X_fe[col])

# Recalculate numeric columns after adding engineered features
num_cols_fe = X_fe.select_dtypes(include=["int64", "float64"]).columns.tolist()

# --------------------------------------------
# 4. Train-test split
# --------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_fe, y, test_size=0.2, random_state=42
)

# --------------------------------------------
# 5. Prepare data for CatBoost
#    - CatBoost can handle numeric NaNs but NOT NaNs in categoricals
#    - Convert categoricals to string + fill missing with "Missing"
# --------------------------------------------
X_train_cb = X_train.copy()
X_test_cb = X_test.copy()

for col in cat_cols:
    if col in X_train_cb.columns:
        X_train_cb[col] = X_train_cb[col].astype(str).fillna("Missing")
        X_test_cb[col] = X_test_cb[col].astype(str).fillna("Missing")

# Categorical feature indices for CatBoost (in the *current* DataFrame)
cat_idx = [X_train_cb.columns.get_loc(c) for c in cat_cols if c in X_train_cb.columns]

print("CatBoost categorical indices:", cat_idx)

# --------------------------------------------
# 6. Optuna objective function for CatBoost
#    - 5-fold CV on training data
# --------------------------------------------
def objective(trial):
    params = {
        "loss_function": "RMSE",
        "random_seed": 42,
        "verbose": False,
        "iterations": trial.suggest_int("iterations", 300, 1500),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 5.0),
        "border_count": trial.suggest_int("border_count", 64, 255),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 200),
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    rmses = []

    for train_idx, valid_idx in kf.split(X_train_cb):
        X_tr = X_train_cb.iloc[train_idx]
        X_val = X_train_cb.iloc[valid_idx]
        y_tr = y_train.iloc[train_idx]
        y_val = y_train.iloc[valid_idx]

        model = CatBoostRegressor(**params)
        model.fit(
            X_tr, y_tr,
            cat_features=cat_idx,
            eval_set=(X_val, y_val),
            use_best_model=True
        )

        preds = model.predict(X_val)
        rmse = mean_squared_error(y_val, preds, squared=False)
        rmses.append(rmse)

    return np.mean(rmses)

# --------------------------------------------
# 7. Run Optuna study to find best hyperparameters
#    - You can increase n_trials for more thorough search
# --------------------------------------------
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("Best trial:")
print(study.best_trial.params)

# --------------------------------------------
# 8. Train final CatBoost model on full training set
#    using best hyperparameters from Optuna
# --------------------------------------------
best_params = study.best_trial.params
best_params.update({
    "loss_function": "RMSE",
    "random_seed": 42,
    "verbose": False
})

best_cat = CatBoostRegressor(**best_params)
best_cat.fit(
    X_train_cb, y_train,
    cat_features=cat_idx,
    eval_set=(X_test_cb, y_test),
    use_best_model=True
)

# --------------------------------------------
# 9. Evaluate on test set
# --------------------------------------------
y_pred_test = best_cat.predict(X_test_cb)
rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)

print(f"\n🔥 Tuned CatBoost Test RMSE: {rmse_test:.4f}")

# --------------------------------------------
# 10. Feature importance (top 25)
# --------------------------------------------
feature_importances = best_cat.get_feature_importance()
fi_df = pd.DataFrame({
    "feature": X_train_cb.columns,
    "importance": feature_importances
}).sort_values("importance", ascending=False)

print("\nTop 25 CatBoost feature importances:")
print(fi_df.head(25))


[I 2025-12-08 00:14:04,183] A new study created in memory with name: no-name-d88e5061-ef76-4f66-abb6-6bd8048fabb4


Categorical columns: ['gender', 'marital_status', 'education_level', 'region', 'employment_status', 'card_type']
Numeric columns: ['customer_id', 'age', 'owns_home', 'has_auto_loan', 'annual_income', 'credit_score', 'credit_limit', 'tenure', 'num_transactions', 'avg_transaction_value', 'online_shopping_freq', 'reward_points_balance', 'travel_frequency', 'utility_payment_count', 'num_children', 'num_credit_cards']
CatBoost categorical indices: [2, 3, 4, 5, 6, 13]


  0%|          | 0/30 [00:00<?, ?it/s]

c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the 

[I 2025-12-08 00:37:47,086] Trial 0 finished with value: 258.14713013478377 and parameters: {'iterations': 1370, 'depth': 7, 'learning_rate': 0.061176893123203936, 'l2_leaf_reg': 4.268145798210141, 'bagging_temperature': 1.8455095303472118, 'border_count': 161, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 57}. Best is trial 0 with value: 258.14713013478377.


...

c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the 

[I 2025-12-08 03:37:00,598] Trial 28 finished with value: 256.3545422617509 and parameters: {'iterations': 1282, 'depth': 4, 'learning_rate': 0.01612751235930232, 'l2_leaf_reg': 4.457107069219993, 'bagging_temperature': 0.590928751693069, 'border_count': 130, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 178}. Best is trial 25 with value: 253.46250044727435.


c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the 

[I 2025-12-08 03:42:00,043] Trial 29 finished with value: 253.46006493182344 and parameters: {'iterations': 1430, 'depth': 5, 'learning_rate': 0.02763204350349723, 'l2_leaf_reg': 2.2076729630825316, 'bagging_temperature': 1.8122912929392145, 'border_count': 174, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 151}. Best is trial 29 with value: 253.46006493182344.
Best trial:
{'iterations': 1430, 'depth': 5, 'learning_rate': 0.02763204350349723, 'l2_leaf_reg': 2.2076729630825316, 'bagging_temperature': 1.8122912929392145, 'border_count': 174, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 151}

🔥 Tuned CatBoost Test RMSE: 252.3193

Top 25 CatBoost feature importances:
                        feature  importance
20                 num_children   19.216545
14             num_transactions    9.759743
13                    card_type    8.576299
26           credit_limit_log1p    8.060238
11                 credit_limit    6.912150
27  avg_transaction_value_log1p    6.493774
9      

c:\Users\athar\anaconda4\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet

# --- Assuming these variables are defined and your model is fitted ---
# 'model' is your fitted ElasticNet instance
# 'X' is your training data DataFrame (used for feature names)
# --------------------------------------------------------------------

# 1. Get the feature names
# Assuming 'poly_transformer' is your fitted PolynomialFeatures instance
# and 'X' is your original feature DataFrame
feature_names = poly.get_feature_names_out(input_features=X_imputed_df.columns)

# 2. Get the estimated coefficients
coefficients = enet.coef_

# 3. Combine them into a DataFrame for easy viewing
coef_summary = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# 4. Display the results
print("### All Feature Coefficients ###")
print(coef_summary)

# 5. Identify features shrunk to ZERO
# The tolerance for zero might be very small, so we use a small threshold
zero_coef_features = coef_summary[np.isclose(coef_summary['Coefficient'], 0, atol=1e-6)]

print("\n### Features Shrunk to Zero (Eliminated) ###")
print(zero_coef_features[zero_coef_features['Coefficient']!=0])

### All Feature Coefficients ###
                                    Feature  Coefficient
0                             gender_female          0.0
1                               gender_male         -0.0
2                    marital_status_married          0.0
3                     marital_status_single         -0.0
4                 education_level_bachelors          0.0
..                                      ...          ...
625           travel_frequency num_children          0.0
626       travel_frequency num_credit_cards          0.0
627      utility_payment_count num_children         -0.0
628  utility_payment_count num_credit_cards          0.0
629           num_children num_credit_cards          0.0

[630 rows x 2 columns]

### Features Shrunk to Zero (Eliminated) ###
                                         Feature   Coefficient
53                     gender_female customer_id  2.104586e-07
86                       gender_male customer_id  1.888470e-07
118           marital_st

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet

# --- Assuming these variables are defined and your model is fitted ---
# 'model' is your fitted ElasticNet instance
# 'X' is your training data DataFrame (used for feature names)
# --------------------------------------------------------------------

# 1. Get the feature names
# Assuming 'poly_transformer' is your fitted PolynomialFeatures instance
# and 'X' is your original feature DataFrame
feature_names = poly.get_feature_names_out(input_features=X_imputed_df.columns)

# 2. Get the estimated coefficients
coefficients = enet.coef_

# 3. Combine them into a DataFrame for easy viewing
coef_summary = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# 4. Display the results
print("### All Feature Coefficients ###")
print(coef_summary)

# 5. Identify features shrunk to ZERO
# The tolerance for zero might be very small, so we use a small threshold
zero_coef_features = coef_summary[np.isclose(coef_summary['Coefficient'], 0, atol=1e-6)]

print("\n### Features Shrunk to Zero (Eliminated) ###")
print(zero_coef_features[zero_coef_features['Coefficient']!=0])

### All Feature Coefficients ###
                                    Feature  Coefficient
0                             gender_female          0.0
1                               gender_male         -0.0
2                    marital_status_married          0.0
3                     marital_status_single         -0.0
4                 education_level_bachelors          0.0
..                                      ...          ...
625           travel_frequency num_children          0.0
626       travel_frequency num_credit_cards          0.0
627      utility_payment_count num_children         -0.0
628  utility_payment_count num_credit_cards          0.0
629           num_children num_credit_cards          0.0

[630 rows x 2 columns]

### Features Shrunk to Zero (Eliminated) ###
                                         Feature   Coefficient
53                     gender_female customer_id  2.104586e-07
86                       gender_male customer_id  1.888470e-07
118           marital_st